# আমাদের প্রয়োজনীয় সব ডিপেন্ডেন্সি গুলা ইনস্টল করে নিচ্ছি; ইনস্টল করার পরে সেশন রিস্টার্ট করার জন্য সাজেস্ট করতে পারে, জাস্ট সিম্পলি ইগনোর।

In [2]:
# Step 1: Install required packages (only needed once)
!pip install --upgrade google-api-python-client google-auth google-auth-oauthlib google-auth-httplib2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.1/216.1 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.38.0
    Uninstalling google-auth-2.38.0:
      Successfully uninstalled google-auth-2.38.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.38.0, but you have google-auth 2.40.3 which is incompatible.


# আমরা আমাদের ড্রাইভারের সাথে কানেক্ট করে সার্ভিসটা ইনেবল করলাম। এজন্য একটা গুগল অথেনটিকেশন পপ-আপ আসতে পারে, প্লিজ সব গুলা পারমিশন দেন।

In [3]:
import io
from google.colab import auth, files
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload

auth.authenticate_user()  # Let user sign in to their Google account
service = build('drive', 'v3')



# এই অংশটুকু কি ভাবে কাজ করছে তার জন্য প্লিজ "README" ফাইল টা read করেন প্লিজ

In [4]:
class DriveOCR:
    def __init__(self, file_list):
        self.file_list = file_list  # List of filenames

    def extract_all_to_single_file(self, output_filename="ocr_output.txt"):
        combined_text = ""

        for file in self.file_list:
            print(f"Processing: {file}")
            mime_type = 'application/vnd.google-apps.document'
            file_metadata = {'name': file, 'mimeType': mime_type}

            uploaded = service.files().create(
                body=file_metadata,
                media_body=MediaFileUpload(file, mimetype='image/jpeg'),
                fields='id'
            ).execute()

            file_id = uploaded.get('id')
            print(f"  Uploaded File ID: {file_id}")

            request = service.files().export_media(fileId=file_id, mimeType='text/plain')
            fh = io.BytesIO()
            downloader = MediaIoBaseDownload(fh, request)
            done = False

            while not done:
                status, done = downloader.next_chunk()

            text = fh.getvalue().decode()
            combined_text += f"\n--- OCR from: {file} ---\n{text}\n"

            # Cleanup from Drive
            service.files().delete(fileId=file_id).execute()

        # Write all OCR text into a single file
        with open(output_filename, "w", encoding="utf-8") as f:
            f.write(combined_text)

        print(f"\n✅ OCR completed. All text saved to '{output_filename}'")
        return output_filename


# step-1: ইমেজ ফাইলে আপলোড করেন এখানে

In [5]:
uploaded = files.upload()  # Upload multiple images here
file_names = list(uploaded.keys())
print("Uploaded Files:", file_names)


Saving bangla.jpg to bangla.jpg
Uploaded Files: ['bangla.jpg']


# step-2: এই কোড ব্লক রান করলে সব টেক্স একটা টেক্সট ফাইল এ জমা হবে

In [6]:
ocr = DriveOCR(file_names)
output_file = ocr.extract_all_to_single_file("combined_ocr.txt")


Processing: bangla.jpg
  Uploaded File ID: 17HVAEtXVL62haqEje6yhOtODVWaM6H9D9hFVFYhTQ7E

✅ OCR completed. All text saved to 'combined_ocr.txt'


# প্রথমবারের জন্য সবগুলো কোড ব্লক একবার রান করতে হবে। কিন্তু এরপর থেকে নতুন ইমেজের জন্য শুধু Step-1 আর Step-2 রিপিট করলেই হবে। আর সব ইমেজের টেক্সট কিন্তু একটাই ফাইলে, 'combined_ocr.txt' এ জমা হবে।